In [86]:
#PigLab3NoteBook

! pip install pandas
! pip install openpyxl
!pip install rdflib
import pandas as pd
import rdflib
import hashlib
import numpy as np
import urllib.parse
from rdflib import Literal, Namespace, RDF, URIRef, Graph, RDFS, OWL
from rdflib.namespace import FOAF, XSD
from rdflib.plugins.sparql import prepareQuery
from pyspark.sql.functions import when, col, lit

### Step 01: Define the file paths and Load the data into Pandas dataframes and Clean

In [87]:
# Define the file paths
files = ['Lab3/Copia de Base Datos DECIDE-INFLUENZA.xlsx',
        'Lab3/Copia de Base Datos DECIDE-ILEITIS.xlsx',
        'Lab3/Copia de Base Datos DECIDE-MYCO HYOPNEUMONIAE.xlsx']

# Load the data into Pandas dataframes
dfs = []
for file in files:
    df = pd.read_excel(file, engine='openpyxl')
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

print(df['PATOGENO'].unique())

['Influenza' 'L. intracellularis' 'M. hyopneumoniae']


In [88]:
df

,ID,PROVINCIA,FECHA,PATOGENO,RESULTADO,MUESTRA,EDAD,GRANJA
0,SV210020,Murcia,05/01/2021,Influenza,Negativo,Hisopos,Cerdas abortadas,Granja de madres
1,210031,Badajoz,05/01/2021,Influenza,Negativo,Pulmon,NaN,Cebadero
2,210071,Girona,08/01/2021,Influenza,Positivo,Fluidos orales,NaN,Cebadero
3,210140,Salamanca,14/01/2021,Influenza,Negativo,Pulmon,NaN,Cebadero
4,210141,Salamanca,14/01/2021,Influenza,Negativo,Pulmon,2.5 meses,Cebadero
...,...,...,...,...,...,...,...,...
1536,SV232121,Zamora,31/08/2023,M. hyopneumoniae,Positivo,Pulmon,8-9 semanas,Granja de madres
1537,234236,Salamanca,19/09/2023,M. hyopneumoniae,Negativo,Pulmon,45-50 dias,Granja de madres
1538,SV232291,Salamanca,22/09/2023,M. hyopneumoniae,Negativo,Pulmon,20 dias postdestete,Granja de madres
1539,SV232385,Murcia,29/09/2023,M. hyopneumoniae,Negativo,Pulmon,NaN,NaN


In [89]:
print(df['RESULTADO'].unique())

['Negativo' 'Positivo' 'Negativo ']


In [90]:
print(df['GRANJA'].unique())

['Granja de madres' 'Cebadero' 'Recría' nan 'Cebo' 'Recria' 'Precebo'
 'Reproductoras' 'Transición' 'Entrada a la explotación' 'Iberico de cebo']


In [91]:
print(df['MUESTRA'].unique())

['Hisopos' 'Pulmon' 'Fluidos orales' 'Lechon' 'Pulmones'
 'Pulmon y ganglios' 'Lechones' 'Hisopo' 'Lechones + hisopos nasales'
 'Fluido oral' 'Hisopo nasal' 'Lenguas lechones' 'Hisopos nasales'
 'Lenguas' 'Hisopos respiratorios' 'Pulmón, gánglios e hisopos'
 'Pulmones + hisopos' 'Pulmón y gánglios' 'Pulmón'
 'Pulmon + fluidos orales' 'Sueros' 'Hisopos de vias respiratorias'
 'Lenguas y colas' 'Pulmon y ganglio' 'Lechón'
 'Cabeza, cuatro patas y pulmon' 'Pulomones' 'Fetos' 'Feto y placenta'
 'Fluidos orales y pulmon' 'Pulmon e hisopos respiratorios'
 'Hisopos de saliva' 'Pulmon con tonsilas' 'Pulmon y tonsilas'
 'Fuidos orales' 'Pulmon y lechon' 'Fluidos orales + hisopos nasales'
 'Lenguas y cordones umbilicales' 'Pulmnes + 1 lechon' 'Pulmon + ganglios'
 'Hisopo pulmonar' 'Pulmones + 1 cabeza'
 'Fluidos orales + hisopos pulmonares + pulmones'
 'Pulmones + fluidos orales' 'Pulmones + 1 fluido oral'
 'Pulmon, corazon y ganglios' 'Pulmones congelados' 'Paquetes pulmonares'
 'Pulmones + 1 h

In [92]:
print(df['EDAD'].unique())

['Cerdas abortadas' nan '2.5 meses' '10 semanas' '9-10 semanas' '2 meses'
 '15-16 semanas' 'Origen madres' 'Cerda reposición' '3 meses' 'Cerda'
 '4 meses' '5, 6 y 7 semanas' '6 meses' '10 - 15 Kg' '7 semanas de vida'
 'Cerdas de recría' 'Cerdas' '9 y 10 semanas' 'Transicion y maternidad'
 'Lechones abortados' 'Cebo' '1 y 4 semanas' 'Lechones'
 'Maternidad y transición' '70 Kg.' '80-90 días de gestación'
 'Lechoneras, transicion, cebo y cerdas' '3, 6 y 9 semanas'
 '25 dias de vida' 'Lechonera y paridera' 'Mitad de cebo' '4 semanas'
 '50 dias de vida' '8-10 Kg.' 'Lechones lactacion' 'Entrada cebo'
 'Cebo 40 Kg.' 'Lechones de 8 Kg' 'Cerda lactante de 14 meses' '30-40 Kg.'
 '22-25 dias' 'Destete' '4-5 semanas de vida' '20 Kg.' '10-11 semanas'
 '4 semanas de vida' '90 Kg' '4 y 10 semanas' '6 semanas de vida'
 '3 y 5 semanas' '6 y 9 semanas' '60 Kg.' '50 días' '5 meses' '48 días'
 '70 días' 'Cerdas Primerizas y Multíparas' '4, 6 y 9 semanas'
 '6-7 semanas de vida' '10 semanas de vida' '5 y 7

In [93]:
# Errors arised with the coding of the Age 
import unicodedata

def clean_text(text):
    if pd.isna(text):
        return None
    text = str(text).strip().lower()
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    return text


# Create a column with the clean text
df['Age_cleaned'] = df['EDAD'].apply(clean_text)
df['GRANJA'] = df['GRANJA'].apply(clean_text)
df['PROVINCIA'] = df['PROVINCIA'].apply(clean_text)

print(df['Age_cleaned'].unique())

['cerdas abortadas' None '2.5 meses' '10 semanas' '9-10 semanas' '2 meses'
 '15-16 semanas' 'origen madres' 'cerda reposicion' '3 meses' 'cerda'
 '4 meses' '5, 6 y 7 semanas' '6 meses' '10 - 15 kg' '7 semanas de vida'
 'cerdas de recria' 'cerdas' '9 y 10 semanas' 'transicion y maternidad'
 'lechones abortados' 'cebo' '1 y 4 semanas' 'lechones'
 'maternidad y transicion' '70 kg.' '80-90 dias de gestacion'
 'lechoneras, transicion, cebo y cerdas' '3, 6 y 9 semanas'
 '25 dias de vida' 'lechonera y paridera' 'mitad de cebo' '4 semanas'
 '50 dias de vida' '8-10 kg.' 'lechones lactacion' 'entrada cebo'
 'cebo 40 kg.' 'lechones de 8 kg' 'cerda lactante de 14 meses' '30-40 kg.'
 '22-25 dias' 'destete' '4-5 semanas de vida' '20 kg.' '10-11 semanas'
 '4 semanas de vida' '90 kg' '4 y 10 semanas' '6 semanas de vida'
 '3 y 5 semanas' '6 y 9 semanas' '60 kg.' '50 dias' '5 meses' '48 dias'
 '70 dias' 'cerdas primerizas y multiparas' '4, 6 y 9 semanas'
 '6-7 semanas de vida' '10 semanas de vida' '5 y 

In [94]:
print(df['PROVINCIA'].unique())

['murcia' 'badajoz' 'girona' 'salamanca' 'burgos' 'zamora' 'navarra'
 'albacete' None 'granada' 'lleida' 'las palmas' 'segovia' 'zaragoza'
 'sevilla' 'barcelona' 'valencia' 'castellon' 'cordoba' 'malaga' 'avila'
 'teruel' 'cadiz' 'toledo' 'tarragona' 'alicante' 'palencia' 'pontevedra'
 'lugo' 'huesca' 'valladolid' 'huelva' 'la coruna' 'extremadura'
 'sierra nevada' 'aldeaseca' 'soria' 'caceres' 'guipuzcoa' 'cuenca'
 'gipuzcoa' 'alava' 'ciudad real' 'bizkaia' 'portugal' 'galicia'
 'catellon' 'islas baleares' 'andalucia' 'a coruna' 'madrid' 'telde']


In [95]:
#Rename Colums

df = df.rename(columns={
    'ID': 'SampleNumber',
    'PROVINCIA': 'Province',
    'FECHA': 'Date',
    'PATOGENO': 'Pathogen',
    'RESULTADO': 'Result',
    'MUESTRA': 'SampleType',
    'Age_cleaned': 'Age', #Age of animal
    'GRANJA': 'FarmType',
})

# Set DiagnosticTest to "PCR" Country to "Spain", Breed to "Meat", and LabReference to "1".
df['DiagnosticTest'] = 'PCR'
df['Country'] = 'Spain'
df['Breed'] = 'Meat'
df["Lab_Reference"] = "3"




In [96]:
df['PigAge'] = df['Age'].astype(str) + ' ' + df['FarmType'].astype(str)

print(df['PigAge'].unique())

['cerdas abortadas granja de madres' 'None cebadero' '2.5 meses cebadero'
 'None granja de madres' '10 semanas cebadero' '9-10 semanas cebadero'
 '2 meses cebadero' '15-16 semanas cebadero'
 'origen madres granja de madres' 'cerda reposicion granja de madres'
 '3 meses cebadero' 'cerda granja de madres' '4 meses cebadero'
 '5, 6 y 7 semanas cebadero' '6 meses cebadero'
 '10 - 15 kg granja de madres' '7 semanas de vida cebadero'
 'cerdas de recria recria' 'cerdas granja de madres'
 '9 y 10 semanas cebadero' 'transicion y maternidad granja de madres'
 'lechones abortados granja de madres' 'cebo cebadero'
 '1 y 4 semanas granja de madres' 'lechones granja de madres'
 'maternidad y transicion granja de madres' '70 kg. cebadero'
 '80-90 dias de gestacion granja de madres'
 'lechoneras, transicion, cebo y cerdas cebadero'
 '3, 6 y 9 semanas cebadero' '25 dias de vida granja de madres'
 'lechonera y paridera granja de madres' 'mitad de cebo cebadero'
 '4 semanas granja de madres' '50 dias de 

In [97]:
#Annotation Mapping

# Add Result mapping for NEG = 0 and POS =1
resultMapping = {
    "Positivo": "1",
    "Negativo": "0",
    "Negativo ": "0",
}
df["Result"] = df["Result"].map(resultMapping).fillna("Missing")

PathogenMapping = {
    "M. hyopneumoniae": "Mycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae antistoffen" : "Mycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae" : "Mycoplasma hyopneumoniae",
    "Influenza" : "Swine influenza virus",
    "Lawsonia intracellularis": "Lawsonia intracellularis",
    'L. intracellularis': "Lawsonia intracellularis",
    
}

# SampleTypeMapping for the additional sample types
SampleTypeMapping = {
    'Pulmón': 'Tissue',
    'Pulmon': 'Tissue',
    'Pulmones': 'Tissue',
    'Lechón': 'Tissue',
    'Lechon': 'Tissue',
    'Lechones': 'Tissue',
    'Lenguas lechones': 'Tissue',
    'Lenguas': 'Tissue',
    'Fluido oral': 'Fluid',
    'Fluidos orales': 'Fluid',
    'Pulmón/ Fluido oral': 'Mixed',
    'Heces': 'Faeces',
    'Raspado traqueal': 'Swab',
    'Hisopo/ Raspado traqueal/ Pulmón': 'Mixed',
    'Intestino': 'Tissue',
    'Hisopo': 'Swab',
    'Hisopos': 'Swab',
    'Hisopos nasales': 'Swab',
    'Raspado traqueal/ Pulmón': 'Mixed',
    'Fluido oral/ Pulmón': 'Mixed',
    'Heces/intestino': 'Mixed',
    'Hisopo rectal': 'Swab',
    'Porción intestinal': 'Tissue',
    'Exudado': 'Fluid',
    'Pulmon y ganglios ': 'Mixed',
    'Líquido pericárdico': 'Fluid',
    'Raspado traqueal/ Fluido oral': 'Mixed',
    'Heces/hisopo rectal': 'Mixed',
    'Hisopo nasal': 'Swab',
    'Hisopo nasal/ Pulmón/ Fluido Oral': 'Mixed',
    ' Lechones/hisopos nasales': 'Mixed',
    'Órgano': 'Tissue',
    'Hisopo intestinal': 'Swab',
    'Feto': 'Tissue',
    'Fetos': 'Tissue',
    'Cabeza, cuatro patas y pulmon': 'Mixed',
    'Hisopo pulmonar': 'Swab',
    'Heces/hisopos': 'Mixed',
    'Ganglio': 'Tissue',
    'Intestino/heces/hisopo': 'Mixed',
    'Pulmón / Fluido Oral': 'Mixed',
    'Pulmón /gánglios e hisopos': 'Mixed',
    'Hisopo fecal': 'Swab',
    'Hisopos de vias respiratorias': 'Swab',
    'Cabeza': 'Tissue',
    'Feto y placenta': 'Mixed',
    'Lenguas y cordones umbilicales': 'Mixed',
    'Pulmnes + 1 lechon': 'Mixed',
    'Sueros': 'Blood',
    'Pulmon y ganglios': 'Mixed',
    'Lechones + hisopos nasales': 'Mixed',
    'Hisopos respiratorios': 'Swab',
    'Pulmón, gánglios e hisopos': 'Mixed',
    'Pulmones + hisopos': 'Mixed',
    'Pulmón y gánglios': 'Mixed',
    'Pulmon + fluidos orales': 'Mixed',
    'Lenguas y colas': 'Mixed',
    'Pulmon y ganglio': 'Mixed',
    'Pulomones': 'Tissue',
    'Fluidos orales y pulmon': 'Mixed',
    'Pulmon e hisopos respiratorios': 'Mixed',
    'Hisopos de saliva': 'Swab',
    'Pulmon con tonsilas': 'Mixed',
    'Pulmon y tonsilas': 'Mixed',
    'Fuidos orales': 'Fluid',
    'Pulmon y lechon': 'Mixed',
    'Fluidos orales + hisopos nasales': 'Mixed',
    'Pulmon + ganglios': 'Mixed',
    'Pulmones + 1 cabeza': 'Mixed',
    'Fluidos orales + hisopos pulmonares + pulmones': 'Mixed',
    'Pulmones + fluidos orales': 'Mixed',
    'Pulmones + 1 fluido oral': 'Mixed',
    'Pulmon, corazon y ganglios': 'Mixed',
    'Pulmones congelados': 'Tissue',
    'Paquetes pulmonares': 'Mixed',
    'Pulmones + 1 hisopo': 'Mixed',
    'Visceras': 'Mixed',
    'Contenido intestinal': 'Faeces',
    'Hisopo y contenido intestinal': 'Mixed',
    'Hisopos ': 'Swab',
    'Hisopos rectales': 'Swab',
    'Heces e hisopos': 'Mixed',
    'Hisopos de heces': 'Swab',
    'Hisopos y contenido intestinal': 'Mixed',
    ' Hisopo rectal': 'Swab',
    'Hisopos y heces': 'Mixed',
    'Heces y contenido intestinal': 'Mixed',
    'Heces ': 'Faeces',
    'Hisopo de heces': 'Swab',
    'Heces e hisopo rectal': 'Mixed',
    'Hisopo digestivo': 'Swab',
    'Hisopos rectales y contenido intestinal': 'Mixed',
    'Hisopos restales': 'Swab',
    'Hisopos digestivos': 'Swab',
    'Pool de heces, hisopos y contenido intestinal': 'Mixed',
    'Hisopos de diarreas': 'Swab',
    'Heces e hisopos rectales': 'Mixed',
    'Hisopos rectales y heces': 'Mixed',
    'Hisopo y heces': 'Mixed',
    'Hisopo rectal y heces': 'Mixed',
    'Heces e hisopo': 'Mixed',
    'Hisopos y heces ': 'Mixed',
    'Hisopos intestinales': 'Swab',
    'Hisopos rectales ': 'Swab',
    'Heces e hisopos intestinales': 'Mixed',
    'Heces e hisopos ': 'Mixed',
    'Heces y ontenido intestinal': 'Mixed',
    'Hisopo de diarrea': 'Swab',
    'Hisopos rectaels': 'Swab',
    'Hisopos de intestino': 'Swab',
    'Hisopos de intestino delgado': 'Swab',
    '4 hisopos rectales': 'Swab',
    '3 hisopos rectales': 'Swab',
    'Raspados nasales': 'Swab',
    'Pulmon e hisopo': 'Mixed',
    'Pulmones e hisopos nasales': 'Mixed',
    'Hisopos vias respiratorias': 'Swab',
}

# Map the SampleType column using the defined mapping
df["SampleType"] = df["SampleType"].map(SampleTypeMapping).fillna("Missing")


df["Pathogen"] = df["Pathogen"].map(PathogenMapping).fillna("Missing")


# Step 1: Convert 'Date' column to datetime format
#df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df['Date'] = pd.to_datetime(df['Date'], format="%d/%m/%Y", errors="coerce")


# Step 2: Format to Year, Month, Day & errors in dates are changed to "Missing"
df['Date'] = df['Date'].dt.strftime("%Y-%m-%d")
df['Date'] = df['Date'].where(df['Date'].notna(), "Missing")


In [98]:
df

,SampleNumber,Province,Date,Pathogen,Result,SampleType,EDAD,FarmType,Age,DiagnosticTest,Country,Breed,Lab_Reference,PigAge
0,SV210020,murcia,2021-01-05,Swine influenza virus,0,Swab,Cerdas abortadas,granja de madres,cerdas abortadas,PCR,Spain,Meat,3,cerdas abortadas granja de madres
1,210031,badajoz,2021-01-05,Swine influenza virus,0,Tissue,NaN,cebadero,None,PCR,Spain,Meat,3,None cebadero
2,210071,girona,2021-01-08,Swine influenza virus,1,Fluid,NaN,cebadero,None,PCR,Spain,Meat,3,None cebadero
3,210140,salamanca,2021-01-14,Swine influenza virus,0,Tissue,NaN,cebadero,None,PCR,Spain,Meat,3,None cebadero
4,210141,salamanca,2021-01-14,Swine influenza virus,0,Tissue,2.5 meses,cebadero,2.5 meses,PCR,Spain,Meat,3,2.5 meses cebadero
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536,SV232121,zamora,2023-08-31,Mycoplasma hyopneumoniae,1,Tissue,8-9 semanas,granja de madres,8-9 semanas,PCR,Spain,Meat,3,8-9 semanas granja de madres
1537,234236,salamanca,2023-09-19,Mycoplasma hyopneumoniae,0,Tissue,45-50 dias,granja de madres,45-50 dias,PCR,Spain,Meat,3,45-50 dias granja de madres
1538,SV232291,salamanca,2023-09-22,Mycoplasma hyopneumoniae,0,Tissue,20 dias postdestete,granja de madres,20 dias postdestete,PCR,Spain,Meat,3,20 dias postdestete granja de madres
1539,SV232385,murcia,2023-09-29,Mycoplasma hyopneumoniae,0,Tissue,NaN,None,None,PCR,Spain,Meat,3,None None


In [99]:
# Province mapping
ProvinceMapping = {
    'a coruna': 'ACoruna',
    'alava': 'Alava',
    'albacete': 'Albacete',
    'aldeaseca': 'Missing',
    'alicante': 'Alicante',
    'andalucia': 'Missing',
    'avila': 'Avila',
    'badajoz': 'Badajoz',
    'barcelona': 'Barcelona',
    'bizkaia': 'Bizkaia',
    'burgos': 'Burgos',
    'caceres': 'Caceres',
    'cadiz': 'Cadiz',
    'castellon': 'Castellon',
    'catellon': 'Castellon',
    'ciudad real': 'CiudadReal',
    'cordoba': 'Cordoba',
    'cuenca': 'Cuenca',
    'extremadura': 'Missing',
    'galicia': 'Missing',
    'gipuzcoa': 'Gipuzkoa',
    'girona': 'Girona',
    'granada': 'Granada',
    'guipuzcoa': 'Gipuzkoa',
    'huelva': 'Huelva',
    'huesca': 'Huesca',
    'islas baleares': 'IslasBaleares',
    'la coruna': 'ACoruna',
    'las palmas': 'LasPalmas',
    'lleida': 'Lleida',
    'lugo': 'Lugo',
    'madrid': 'Madrid',
    'malaga': 'Malaga',
    'murcia': 'Murcia',
    'navarra': 'Navarre',
    'None': 'Missing',
    'palencia': 'Palencia',
    'pontevedra': 'Pontevedra',
    'portugal': 'Portugal',
    'salamanca': 'Salamanca',
    'segovia': 'Segovia',
    'sevilla': 'Sevilla',
    'sierra nevada': 'Granada',
    'soria': 'Soria',
    'tarragona': 'Tarragona',
    'telde': 'GranCanaria',
    'teruel': 'Teruel',
    'toledo': 'Toledo',
    'valencia': 'Valencia',
    'valladolid': 'Valladolid',
    'zamora': 'Zamora',
    'zaragoza': 'Zaragoza',

}
df["Province"] = df["Province"].map(ProvinceMapping).fillna("Missing")



In [100]:
# PigAge mapping
PigStageMapping = {
    'cerdas abortadas granja de madres': 'Sow',
'None cebadero': 'Finishing',
'2.5 meses cebadero': 'Finishing',
'None granja de madres': 'Sow',
'10 semanas cebadero': 'Nursery',
'9-10 semanas cebadero': 'Nursery',
'2 meses cebadero': 'Finishing',
'15-16 semanas cebadero': 'Finishing',
'origen madres granja de madres': 'Sow',
'cerda reposicion granja de madres': 'Gilt',
'3 meses cebadero': 'Finishing',
'cerda granja de madres': 'Sow',
'4 meses cebadero': 'Finishing',
'5, 6 y 7 semanas cebadero': 'Nursery',
'6 meses cebadero': 'Finishing',
'10 - 15 kg granja de madres': 'Nursery',
'7 semanas de vida cebadero': 'Nursery',
'cerdas de recria recria': 'Gilt',
'cerdas granja de madres': 'Sow',
'9 y 10 semanas cebadero': 'Nursery',
'transicion y maternidad granja de madres': 'Mixed',
'lechones abortados granja de madres': 'Sow',
'cebo cebadero': 'Finishing',
'1 y 4 semanas granja de madres': 'Suckling',
'lechones granja de madres': 'Suckling',
'maternidad y transicion granja de madres': 'Mixed',
'70 kg. cebadero': 'Finishing',
'80-90 dias de gestacion granja de madres': 'Sow',
'lechoneras, transicion, cebo y cerdas cebadero': 'Mixed',
'3, 6 y 9 semanas cebadero': 'Nursery',
'25 dias de vida granja de madres': 'Weaning',
'lechonera y paridera granja de madres': 'Mixed',
'mitad de cebo cebadero': 'Finishing',
'4 semanas granja de madres': 'Weaning',
'50 dias de vida cebadero': 'Nursery',
'8-10 kg. granja de madres': 'Nursery',
'lechones lactacion granja de madres': 'Suckling',
'entrada cebo cebadero': 'Finishing',
'cebo 40 kg. cebadero': 'Finishing',
'lechones de 8 kg granja de madres': 'Nursery',
'cerda lactante de 14 meses granja de madres': 'Sow',
'30-40 kg. cebadero': 'Finishing',
'22-25 dias granja de madres': 'Weaning',
'destete cebadero': 'Weaning',
'4-5 semanas de vida cebadero': 'Nursery',
'20 kg. granja de madres': 'Finishing',
'10-11 semanas cebadero': 'Finishing',
'4 semanas de vida cebadero': 'Weaning',
'90 kg cebadero': 'Finishing',
'4 y 10 semanas cebadero': 'Mixed',
'6 semanas de vida cebadero': 'Nursery',
'lechones cebadero': 'Finishing',
'3 y 5 semanas granja de madres': 'Mixed',
'6 y 9 semanas cebadero': 'Nursery',
'60 kg. cebadero': 'Finishing',
'50 dias cebadero': 'Nursery',
'5 meses cebadero': 'Finishing',
'48 dias cebadero': 'Nursery',
'70 dias cebadero': 'Nursery',
'cerdas primerizas y multiparas granja de madres': 'Sow',
'4, 6 y 9 semanas cebadero': 'Nursery',
'6-7 semanas de vida cebadero': 'Nursery',
'10 semanas de vida cebadero': 'Nursery',
'5 y 7 semanas cebadero': 'Nursery',
'8 semabas cebadero': 'Nursery',
'lactacion granja de madres': 'Suckling',
'cerda primer parto granja de madres': 'Sow',
'lactacion y gestantes granja de madres': 'Mixed',
'transicion granja de madres': 'Nursery',
'gestante granja de madres': 'Sow',
'None None': 'Missing',
'5.5 meses cebadero': 'Finishing',
'nulipara y multiparas granja de madres': 'Mixed',
'120 dias cebadero': 'Finishing',
'24 semanas (cebo 100 kg.) cebadero': 'Finishing',
'22 kg. granja de madres': 'Finishing',
'6-7 semanas cebadero': 'Nursery',
'100 kg. cebadero': 'Finishing',
'45-50 dias de vida cebadero': 'Nursery',
'cerdas reposicion granja de madres': 'Gilt',
'lechones 1-6 dias granja de madres': 'Suckling',
'lechones lactantes granja de madres': 'Suckling',
'90 dias de vida cebadero': 'Finishing',
'3.5 - 4 meses cebadero': 'Finishing',
'None cebo': 'Finishing',
'lechones cebo': 'Finishing',
'80 kg cebo': 'Finishing',
'100 kg cebo': 'Finishing',
'4 meses cebo': 'Finishing',
'3 meses cebo': 'Finishing',
'destete cebo': 'Weaning',
'None recria': 'Gilt',
'11 semanas cebo': 'Finishing',
'cebo cebo': 'Finishing',
'reproductoras granja de madres': 'Sow',
'6 semanas cebo': 'Finishing',
'lechon cebo': 'Finishing',
'6 meses cebo': 'Finishing',
'50 kg cebo': 'Finishing',
'5 semanas cebo': 'Nursery',
'None precebo': 'Nursery',
'40-100 kg cebo': 'Finishing',
'transicion cebo': 'Mixed',
'1.5 meses cebo': 'Nursery',
'5 dias granja de madres': 'Suckling',
'10-12 semanas cebo': 'Finishing',
'8-10 semanas cebo': 'Nursery',
'50-60 kg cebo': 'Finishing',
'6 semanas granja de madres': 'Nursery',
'cerdos cebo': 'Finishing',
'40 kg. cebo': 'Finishing',
'adultas reproductoras': 'Sow',
'25 dias granja de madres': 'Weaning',
'cerdas primerizas granja de madres': 'Sow',
'7 meses cebo': 'Finishing',
'30 kg cebo': 'Finishing',
'2-3 meses cebo': 'Finishing',
'5 meses cebo': 'Finishing',
'3-4 semanas cebo': 'Weaning',
'1.5-2 meses cebo': 'Nursery',
'lechones recria': 'Gilt',
'1 mes en cebadero cebo': 'Finishing',
'lechones transicion': 'Nursery',
'adultos cebo': 'Finishing',
'30-40-kg cebo': 'Finishing',
'1-3 meses cebo': 'Finishing',
'cerdas adultas granja de madres': 'Sow',
'6 meses granja de madres': 'Other',
'lechones destete granja de madres': 'Weaning',
'50 dias cebo': 'Nursery',
'2 meses cebo': 'Nursery',
'12-25 kg. cebo': 'Mixed',
'4.5 meses cebo': 'Finishing',
'4-5 meses cebo': 'Finishing',
'15 kg cebo': 'Nursery',
'lechones de entrada a cebo cebo': 'Finishing',
'3.5 meses cebo': 'Finishing',
'2.5 meses cebo': 'Finishing',
'10 semanas cebo': 'Nursery',
'4 meses en cebo cebo': 'Finishing',
'1.5 meses en cebo cebo': 'Finishing',
'cerdos de engorde cebo': 'Finishing',
'lechones al destete granja de madres': 'Weaning',
'entrada a cebadero cebo': 'Finishing',
'lechones postdestete granja de madres': 'Nursery',
'20 kg cebo': 'Finishing',
'5 semanas granja de madres': 'Nursery',
'35 kg. cebo': 'Finishing',
'lechones transicion cebo': 'Nursery',
'60 kg. cebo': 'Finishing',
'16 semanas cebo': 'Finishing',
'1-2 anos granja de madres': 'Sow',
'20 semanas cebo': 'Finishing',
'30 kg. cebo': 'Finishing',
'60-70 kg. cebo': 'Finishing',
'25-30 kg. cebo': 'Finishing',
'lechones entrada a la explotacion': 'Other',
'30 dias post destete cebo': 'Nursery',
'60 dias cebo': 'Nursery',
'15 semanas cebo': 'Finishing',
'5-6 meses cebo': 'Finishing',
'engorde cebo': 'Finishing',
'4 meses iberico de cebo': 'Finishing',
'1 mes en cebo iberico de cebo': 'Finishing',
'lechon granja de madres': 'Suckling',
'cerda adulta granja de madres': 'Sow',
'lechones entrada a cebo iberico de cebo': 'Finishing',
'lechones en paridera granja de madres': 'Suckling',
'8 semanas cebo': 'Nursery',
'7 semanas cebo': 'Nursery',
'3-4 meses cebo': 'Finishing',
'None iberico de cebo': 'Finishing',
'15 dias destetados cebo': 'Nursery',
'recria granja de madres': 'Gilt',
'3,5 meses cebo': 'Finishing',
'14 semanas cebo': 'Finishing',
'30-40 kg. cebo': 'Finishing',
'cerda gestante granja de madres': 'Sow',
'4,5 meses cebo': 'Finishing',
'cerdas recria granja de madres': 'Gilt',
'1-2 meses cebo': 'Nursery',
'post destete granja de madres': 'Nursery',
'hembras granja de madres': 'Sow',
'3,5-4,5 meses cebo': 'Finishing',
'4 anos granja de madres': 'Sow',
'5-11 semanas cebo': 'Nursery',
'35-40 dias cebo': 'Nursery',
'cerdas nuliparas granja de madres': 'Gilt',
'70-80 kg. cebo': 'Finishing',
'40-50 kg. cebo': 'Finishing',
'7-8 meses cebo': 'Finishing',
'destetados granja de madres': 'Weaning',
'lechon destetado granja de madres': 'Weaning',
'100 kg. cebo': 'Finishing',
'15 dias en cebadero cebo': 'Finishing',
'10-15 dias en cebadero cebo': 'Finishing',
'10 dias post destete cebo': 'Nursery',
'50-60 dias cebo': 'Nursery',
'cerdas multiparas granja de madres': 'Sow',
'17 semanas cebo': 'Finishing',
'100 kg. (6 meses) cebo': 'Finishing',
'5 - 5.5 meses cebo': 'Finishing',
'4 - 5 meses cebo': 'Finishing',
'4-5 semanas granja de madres': 'Nursery',
'1 semana post destete cebo': 'Nursery',
'50 kg. cebo': 'Finishing',
'2-5 meses cebo': 'Finishing',
'45-60 dias cebo': 'Nursery',
'80 kg. cebo': 'Finishing',
'45 dias cebo': 'Nursery',
'10-15 kg. granja de madres': 'Nursery',
'14 meses granja de madres': 'Sow',
'25 kg. cebo': 'Finishing',
'130 kg. cebo': 'Finishing',
'lechon de transicion granja de madres': 'Nursery',
'3 meses precebo': 'Finishing',
'20 kg. cebo': 'Finishing',
'1 mes granja de madres': 'Weaning',
'4-4,5 meses cebo': 'Finishing',
'lechon postdestete cebo': 'Nursery',
'5-6 semanas cebo': 'Nursery',
'15 dias post destete cebo': 'Nursery',
'35 dias cebo': 'Nursery',
'8 - 9 semanas cebo': 'Nursery',
'lechones post destete cebo': 'Nursery',
'7-8 semanas cebo': 'Nursery',
'lechones transicion granja de madres': 'Nursery',
'2,5 meses cebo': 'Finishing',
'5 semanas recria': 'Gilt',
'6-12 semanas cebo': 'Mixed',
'cerda en lactacion granja de madres': 'Sow',
'lechones con menos de un dia en cebo cebo': 'Finishing',
'3 meses post destete cebo': 'Finishing',
'3 meses de cebo cebo': 'Finishing',
'45-55 kg. cebo': 'Finishing',
'6-7 semanas cebo': 'Nursery',
'4-4.5 mese de vida cebo': 'Finishing',
'140 dias cebo': 'Finishing',
'1 mes destetados cebo': 'Nursery',
'15-20 kg cebo': 'Nursery',
'final de cebo cebo': 'Finishing',
'110 dias cebo': 'Finishing',
'8 meses cebo': 'Finishing',
'70 dias cebo': 'Nursery',
'4,5 semanas cebo': 'Nursery',
'primalas ibericos granja de madres': 'Gilt',
'9 semanas cebo': 'Nursery',
'3 meses en cebo cebo': 'Finishing',
'7 dias post destete cebo': 'Nursery',
'cerdos mitad y final de cebo cebo': 'Finishing',
'lechon sala transicion cebo': 'Nursery',
'lechones de paridea y gestacion granja de madres': 'Mixed',
'2-4 meses cebo': 'Finishing',
'2.5-4 meses cebo': 'Finishing',
'2.5-5.5 meses cebo': 'Finishing',
'1.5 - 2.5 meses granja de madres': 'Nursery',
'8-9 semanas cebo': 'Nursery',
'4-6 semanas granja de madres': 'Nursery',
'12 semanas cebo': 'Finishing',
'20 dias post destete cebo': 'Nursery',
'28 dias granja de madres': 'Weaning',
'3 semanas post destete cebo': 'Nursery',
'precebo cebo': 'Nursery',
'6-7 meses cebo': 'Finishing',
'lechones destetados cebo': 'Weaning',
'post destete cebo': 'Nursery',
'9-10 semanas cebo': 'Nursery',
'60 kg cebo': 'Finishing',
'30-40 kg cebo': 'Finishing',
'48 dias granja de madres': 'Nursery',
'8-9 semanas granja de madres': 'Nursery',
'45-50 dias granja de madres': 'Nursery',
'20 dias postdestete granja de madres': 'Nursery',

}

# Map the 'Edad' column using the English translation mapping
#df['PigStages_Age'] = df['Age'].str.lower().map(PigStageMapping)
df["PigStage"] = df["PigAge"].map(PigStageMapping).fillna("Missing")

# Combine the results from both mappings (PigStages_Age and PigStages_FarmType)
#df['PigStages'] = df.apply(lambda x: x['PigStages_Age'] if pd.notna(x['PigStages_Age']) else x['PigStages_FarmType'], axis=1)
#df['PigStage'] = df.apply(lambda x: x['PigStages_Age'] if pd.notna(x['PigStages_Age']) else x['PigStages_FarmType'], axis=1).fillna("Missing")

# Drop intermediate columns if needed
#df.drop(['PigStages_Age'], axis=1, inplace=True)
df = df.drop(['Age', 'EDAD'], axis=1)
df = df.drop(['FarmType'], axis=1)

In [101]:
df = df.drop(['PigAge'], axis=1)
df.head(50)

,SampleNumber,Province,Date,Pathogen,Result,SampleType,DiagnosticTest,Country,Breed,Lab_Reference,PigStage
0,SV210020,Murcia,2021-01-05,Swine influenza virus,0,Swab,PCR,Spain,Meat,3,Sow
1,210031,Badajoz,2021-01-05,Swine influenza virus,0,Tissue,PCR,Spain,Meat,3,Finishing
2,210071,Girona,2021-01-08,Swine influenza virus,1,Fluid,PCR,Spain,Meat,3,Finishing
3,210140,Salamanca,2021-01-14,Swine influenza virus,0,Tissue,PCR,Spain,Meat,3,Finishing
4,210141,Salamanca,2021-01-14,Swine influenza virus,0,Tissue,PCR,Spain,Meat,3,Finishing
5,SV210121,Burgos,2021-01-19,Swine influenza virus,0,Tissue,PCR,Spain,Meat,3,Finishing
6,SV210179,Zamora,2021-01-21,Swine influenza virus,0,Swab,PCR,Spain,Meat,3,Sow
7,SV210233,Navarre,2021-01-26,Swine influenza virus,1,Tissue,PCR,Spain,Meat,3,Finishing
8,SV210252,Albacete,2021-01-27,Swine influenza virus,0,Tissue,PCR,Spain,Meat,3,Finishing
9,SV210330,Missing,2021-02-03,Swine influenza virus,1,Tissue,PCR,Spain,Meat,3,Nursery


In [102]:
print(df['Pathogen'].unique())

['Swine influenza virus' 'Lawsonia intracellularis'
 'Mycoplasma hyopneumoniae']


In [103]:
print(df['Result'].unique())

['0' '1']


In [104]:
print(df['SampleType'].unique())

['Swab' 'Tissue' 'Fluid' 'Mixed' 'Blood' 'Faeces']


In [105]:
print(df['PigStage'].unique())

['Sow' 'Finishing' 'Nursery' 'Gilt' 'Mixed' 'Suckling' 'Weaning' 'Missing'
 'Other']


In [106]:
# Save to desired path
df.to_csv('output/PigLab3_update20250109.csv', index=False)

In [107]:
df_all = df

### Step 02: Create an RDF graph and namespaces.

In [108]:
g = rdflib.Graph()
xsd = Namespace('http://www.w3.org/2001/XMLSchema#')
g.bind('xsd', xsd)

# Define your custom namespace for your ontology's properties
LHO = Namespace("http://www.purl.org/decide/LiveStockHealthOnto/LHO#")
g.bind('LHO', LHO)

SKOS = Namespace("http://www.w3.org/2004/02/skos/core#")
g.bind('skos', SKOS)

decide=  Namespace("http://www.purl.org/decide#")
g.bind('decide', decide)
ncit = Namespace("http://purl.obolibrary.org/obo/NCIT_C25464")
g.bind('ncit', ncit)
agrovoc = Namespace ("http://aims.fao.org/aos/agrovoc")
g.bind('agrovoc', agrovoc)

###  Step 03: Iterate over the Panda DataFrame and map to ontology properties:

In [109]:
# for slugging local names or avoid spaces in names to create URIs along with label names

import re
import urllib.parse

def _safe_local_name(v):
    v = str(v).strip()
    # RFC3986 unreserved: A-Z a-z 0-9 - . _ ~
    v = re.sub(r'[^A-Za-z0-9\-._~]', '_', v)
    v = re.sub(r'_+', '_', v)  # collapse repeated underscores (optional)
    return v

In [110]:
#For Pig Data frame RDF conversion
#for index, row in barometer.head(500).iterrows():
##................................><.........................................................................
##Note: spaces in URIs are not OK. RDFLib (and the RDF specs) require valid URIs; a space makes them invalid. 
##If there is space in names then got errors in the Turtle serialization.
##................................><.........................................................................
for index, row in df_all.iterrows():
    SampleNumber = row["SampleNumber"]
    Date = row["Date"]
    Pathogen = row["Pathogen"]
    Breed = row["Breed"]
    Country = row["Country"]
    Province = row["Province"]
    SampleType = row["SampleType"]
    SampleResult = row["Result"]
    DiagnosticTest = row["DiagnosticTest"]
    PigStage= row["PigStage"]

    
    # Create a unique URI for each sample based on the row index
    PigSample_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#Lab1PigSample_{index}")
    
    # Add sample type assertion
    g.add((PigSample_uri, RDF.type, LHO.PigSample))
    description = "An individual representing a sample from a Pig and Piglets."
    g.add((PigSample_uri, RDFS.comment, Literal(description, lang="en")))

    
    #if "Pathogen" in row and row["Pathogen"]:    # Making little changes to pathogen URI so that it will not create error
        #Pathogen = row["Pathogen"]
        #Pathogen_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Pathogen}")
        #g.add((PigSample_uri, LHO.hasPathogen, Pathogen_uri))
        #g.add((Pathogen_uri, RDF.type, decide.Pathogen))
        #description = "An individual representing Pig pathogen."
        #g.add((Pathogen_uri, RDFS.comment, Literal(description, lang="en")))

    if "Pathogen" in row and row["Pathogen"]:
        Pathogen = str(row["Pathogen"]).strip()
        Pathogen_local = _safe_local_name(Pathogen)  # e.g., "Swine_influenza_virus"
        Pathogen_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Pathogen_local}")
        g.add((PigSample_uri, LHO.hasPathogen, Pathogen_uri))
        g.add((Pathogen_uri, RDF.type, decide.Pathogen))
        description = "An individual representing Pig pathogen."
        g.add((Pathogen_uri, RDFS.comment, Literal(description, lang="en")))
        g.add((Pathogen_uri, RDFS.label, Literal(Pathogen)))  # human-readable conventional label, e.g., "Swine influenza virus" same thing we can do with other properties like breed and diagnostic test etc. if we have sapce

        
    if "Breed" in row and row["Breed"]:
        Breed = row["Breed"]
        Breed_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Breed}")
        g.add((PigSample_uri, LHO.hasBreed, Breed_uri))
        g.add((Breed_uri, RDF.type, LHO.Breed))
        description = "An individual representing a Piglet breed which is Meat."
        g.add((Breed_uri, RDFS.comment, Literal(description, lang="en")))

    if "Lab_Reference" in row and row["Lab_Reference"]:  #Adding that code
        Lab_Reference = row["Lab_Reference"]
        Lab_Reference_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Lab_Reference}")
        g.add((PigSample_uri, LHO.hasLab_Reference, Lab_Reference_uri))
        g.add((Lab_Reference_uri, RDF.type, LHO.Lab_Reference))
        description = "An individual representing from which lab refence it belongs."
        g.add((Lab_Reference_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "DiagnosticTest" in row and row["DiagnosticTest"]:
        DiagnosticTest= row["DiagnosticTest"]
        DiagnosticTest_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{DiagnosticTest}")
        g.add((PigSample_uri, LHO.hasDiagnosticTest, DiagnosticTest_uri))
        g.add((DiagnosticTest_uri, RDF.type, LHO.DiagnosticTest))
    if DiagnosticTest == "PCR":
        description = "An individual representing DNA/RNA amplification for rapid pathogen detection."
    elif SampleType == "Culture":
        description = "An individual representing a Growing live microorganisms for identification."
        g.add((DiagnosticTest_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Country" in row and row["Country"]:
        Country = row["Country"].strip()  # Use strip() to remove leading and trailing spaces
        # Encode the Country value to create a valid URI
        Country = urllib.parse.quote(Country)
        Country_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Country}")
    
        # Map to the specific class URI in your ontology and add the label
        g.add((PigSample_uri, LHO.hasCountry, Country_uri))
        g.add((Country_uri, RDF.type, URIRef("http://purl.obolibrary.org/obo/NCIT_C25464")))  # Use the specific class URI
        g.add((Country_uri, RDFS.label, Literal("Spain")))
        description = "An individual representing different Countries."
        g.add((Country_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Province" in row and row["Province"]:
        Province = row["Province"].strip()
        Province = urllib.parse.quote(Province)
        Province_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Province}")
        g.add((PigSample_uri, LHO.hasProvince, Province_uri))
        g.add((Province_uri, RDF.type, LHO.Province))
        description = "An individual representing different Province."
        g.add((Province_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "PigStage" in row and row["PigStage"]:
        Stage = row["PigStage"]
        Stage_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Stage}")
        g.add((PigSample_uri, LHO.hasProductionStages, Stage_uri))
        g.add((Stage_uri, RDF.type, LHO.PigProductionStage))
        description = "An individual representing Production stage ."
        g.add((Stage_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "SampleType" in row and row["SampleType"]:
        SampleType = row["SampleType"]
        SampleType_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleType}")
        # Add statements for SampleType
        g.add((PigSample_uri, LHO.hasSampleType, SampleType_uri))
        g.add((SampleType_uri, RDF.type, URIRef("http://www.purl.org/decide#SampleType")))
    if SampleType == "Autopsy":
        description = "An individual representing a cattle sample obtained through autopsy."
    elif SampleType == "BAL":
        description = "An individual representing a cattle sample obtained through bronchoalveolar lavage (BAL)."
    elif SampleType == "SWAB":
        description = "An individual representing a cattle sample obtained through swabbing."
    elif SampleType == "Missing":
        description = "An individual representing a missing or unspecified cattle sample type."
        g.add((SampleType_uri, RDFS.comment, Literal(description, lang="en")))
           
    if "Result" in row and row["Result"]:
        SampleResult = row["Result"]
        # Create a unique URI for the Result based on the value
        Result_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleResult}")
        g.add((PigSample_uri, LHO.hasResult, Result_uri))
        g.add((Result_uri, RDF.type, decide.SampleResult))
    if SampleResult == "1":
        description = "An individual representing a Positive test result"
    elif SampleResult == "0":
        description = "An individual representing a negative test result"
    elif SampleResult == "missing":
        description = "An individual representing an unknown or missing test result."
    g.add((Result_uri, RDFS.comment, Literal(description, lang="en")))


    if "Date" in row and not pd.isnull(row["Date"]):  # Check for NaN or None values
        Date = str(row["Date"]).strip()
    
    # Split the datetime string and take the date part
        Date = Date.split()[0]
    
    # Create a unique URI for the Date based on the value (use the Date directly)
        Date_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Date}")
    
    # Add Date assertion
        g.add((PigSample_uri, LHO.hasDate, Date_uri))
        g.add((Date_uri, RDF.type, LHO.Date))
    

# Serialize the RDF graph to a file
rdf_output_file = "output/PigLab1.rdf"
g.serialize(rdf_output_file, format="xml")

# Serialize the RDF graph to Turtle format and print it too see RDF conversion
#turtle_data = g.serialize(format="turtle")
#print(turtle_data)

<Graph identifier=N3d94c013ccdf43e3ad437751f15d49c1 (<class 'rdflib.graph.Graph'>)>

### Step 4: For Knowledge Graph,Load the RDF data and ontology into a Panda DataFrame: 

In [111]:
# Parse the ontology file in OWL format and add it to the graph
path_to_ontology = "Ontology/LivestockHealthOntology.owl"
g.parse(path_to_ontology, format="xml")

<Graph identifier=N3d94c013ccdf43e3ad437751f15d49c1 (<class 'rdflib.graph.Graph'>)>

### Step 05: Query the data from updated ontology 

In [112]:
query = """
PREFIX rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX decide: <http://www.purl.org/decide#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX LHO:   <http://www.purl.org/decide/LiveStockHealthOnto/LHO#>
PREFIX skos:  <http://www.w3.org/2004/02/skos/core#>
PREFIX xsd:   <http://www.w3.org/2001/XMLSchema#>

SELECT
  (STRAFTER(STR(?Sample), "#") AS ?SampleNumber)
  (STRAFTER(STR(?Province), "#") AS ?Province)
  (?FlooredDate AS ?FlooredDate)
  (COALESCE(STR(?PathogenLabel), REPLACE(STRAFTER(STR(?Pathogen), "#"), "_", " ")) AS ?Pathogen)
  (STRAFTER(STR(?SampleResult), "#") AS ?Result)
  (STRAFTER(STR(?SampleType), "#") AS ?SampleType)
  (STRAFTER(STR(?DiagnosticTest), "#") AS ?DiagnosticTest)
  (STRAFTER(STR(?Country), "#") AS ?Country)
  (STRAFTER(STR(?Breed), "#") AS ?Breed)
  (STRAFTER(STR(?Lab_Reference), "#") AS ?Lab_Reference)
  (STRAFTER(STR(?PigProductionStages), "#") AS ?PigStage)
  (STRAFTER(STR(?AutonomousCommunity), "#") AS ?AutonomousCommunity)
  (?MonthNum AS ?Month)
  (?YearNum  AS ?Year)
WHERE {
  ?Sample rdf:type LHO:PigSample .
  ?Sample LHO:hasDate ?Date .
  ?Sample LHO:hasPathogen ?Pathogen .
  ?Sample LHO:hasBreed ?Breed .
  ?Sample LHO:hasSampleType ?SampleType .
  ?Sample LHO:hasDiagnosticTest ?DiagnosticTest .
  ?Sample LHO:hasProductionStages ?PigProductionStages .
  ?Sample LHO:hasProvince ?Province .
  ?Sample LHO:hasCountry ?Country .
  ?Sample LHO:hasResult ?SampleResult .
  ?Sample LHO:hasLab_Reference ?Lab_Reference .
  OPTIONAL { ?Sample LHO:hasAutonomousCommunity ?AutonomousCommunity . }

  # Pathogen label only (prefer rdfs:label/skos:prefLabel; prefer en or no-lang)
  OPTIONAL {
    ?Pathogen (rdfs:label|skos:prefLabel) ?PathogenLabel .
    FILTER(LANG(?PathogenLabel) = "" || LANGMATCHES(LANG(?PathogenLabel), "en"))
  }

  # ---- Date parsing (supports DD/MM/YYYY and YYYY-MM-DD) ----
  BIND( STRAFTER(STR(?Date), "#") AS ?DateStr )
  BIND( ?DateStr AS ?FlooredDate )

  # month
  BIND(
    xsd:integer(
      IF(CONTAINS(?DateStr, "/"),
         SUBSTR(?DateStr, 4, 2),   # DD/MM/YYYY -> MM is positions 4-5
         SUBSTR(?DateStr, 6, 2)    # YYYY-MM-DD -> MM is positions 6-7
      )
    ) AS ?MonthNum
  )

  # year
  BIND(
    xsd:integer(
      IF(CONTAINS(?DateStr, "/"),
         SUBSTR(?DateStr, 7, 4),   # DD/MM/YYYY -> YYYY starts at 7
         SUBSTR(?DateStr, 1, 4)    # YYYY-MM-DD -> YYYY is positions 1-4
      )
    ) AS ?YearNum
  )
}
"""
results = g.query(query)

data = []
for row in results:
    data.append(list(row))

df = pd.DataFrame(data, columns=[
    "SampleNumber","Province","FlooredDate","Pathogen","Result","SampleType",
    "DiagnosticTest","Country","Breed","LabReference","PigStage",
    "AutonomousCommunity","Month","Year"
])

# Display the dataframe
df

,SampleNumber,Province,FlooredDate,Pathogen,Result,SampleType,DiagnosticTest,Country,Breed,LabReference,PigStage,AutonomousCommunity,Month,Year
0,Lab1PigSample_0,Murcia,2021-01-05,Swine influenza virus,0,Swab,PCR,Spain,Meat,3,Sow,None,1,2021
1,Lab1PigSample_1,Badajoz,2021-01-05,Swine influenza virus,0,Tissue,PCR,Spain,Meat,3,Finishing,None,1,2021
2,Lab1PigSample_2,Girona,2021-01-08,Swine influenza virus,1,Fluid,PCR,Spain,Meat,3,Finishing,None,1,2021
3,Lab1PigSample_3,Salamanca,2021-01-14,Swine influenza virus,0,Tissue,PCR,Spain,Meat,3,Finishing,None,1,2021
4,Lab1PigSample_4,Salamanca,2021-01-14,Swine influenza virus,0,Tissue,PCR,Spain,Meat,3,Finishing,None,1,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536,Lab1PigSample_1536,Zamora,2023-08-31,Mycoplasma hyopneumoniae,1,Tissue,PCR,Spain,Meat,3,Nursery,None,8,2023
1537,Lab1PigSample_1537,Salamanca,2023-09-19,Mycoplasma hyopneumoniae,0,Tissue,PCR,Spain,Meat,3,Nursery,None,9,2023
1538,Lab1PigSample_1538,Salamanca,2023-09-22,Mycoplasma hyopneumoniae,0,Tissue,PCR,Spain,Meat,3,Nursery,None,9,2023
1539,Lab1PigSample_1539,Murcia,2023-09-29,Mycoplasma hyopneumoniae,0,Tissue,PCR,Spain,Meat,3,Missing,None,9,2023


In [113]:
# Save Results: save to CSV
import os
os.makedirs("output", exist_ok=True)

df.to_csv("output/PigLab3_query_results.csv", index=False, encoding="utf-8")
print("Saved: output/PigLab3_query_results_baromterepig.csv")

Saved: output/PigLab3_query_results_baromterepig.csv
